In [ ]:
# 1. Tải về bộ dữ liệu
import os
import random
import csv
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from PIL import Image # type: ignore

import time

import torch # type: ignore
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
import torch.nn.functional as F # type: ignore
import torch.utils.data as data # type: ignore

import torchvision.transforms as transform # type: ignore
import torchvision.datasets as datasets # type: ignore

from torchsummary import summary # type: ignore

ROOT = '/data'
train_data = datasets.MNIST(root=ROOT, train=True, download=True)
test_data = datasets.MNIST(root=ROOT, train=False, download=True)

In [ ]:
# 2. Tiền xử lý dữ liệu

# Split training : validation = 0.9 : 0.1
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

# compute mean and std for normalization
mean = train_data.dataset.data.float().mean() / 255
std = train_data.dataset.data.float().std() / 255

train_transforms = transform.Compose([
    transform.ToTensor(),
    transform.Normalize(mean=[mean], std=[std])
])

test_transforms = transform.Compose([
    transform.ToTensor(),
    transform.Normalize(mean=[mean], std=[std])
])

train_data.dataset.transform = train_transforms
valid_data.dataset.transform = test_transforms

# Create dataloader
BATCH_SIZE = 256

train_dataloader = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE,
                                 num_workers=0)

valid_dataloader = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE,
                                 num_workers=0)

In [21]:
# 3. Xây dựng mô hình LeNet
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding='same')
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc1(outputs)
        outputs = self.fc2(outputs)
        outputs = self.fc3(outputs)
        return outputs

In [ ]:
# 4. Huấn luyện mô hình
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()
    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(inputs)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # update parameters
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches"
                " | accuracy {:8.3f}".format
                                            (epoch, idx, len(train_dataloader), total_acc/total_count)
                )
            total_acc, total_count = 0, 0
            start_time = time.time()
    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_loss, epoch_acc

# Evaluation function
def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device) # type: ignore
            labels = labels.to(device) # type: ignore

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_loss, epoch_acc


In [ ]:
# Training
num_classes = len(train_data.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
lenet_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet_model.parameters(), lr=0.001, weight_decay=0.0001)

num_epochs = 10
save_model = './model'
# Create the directory if it doesn't exist
os.makedirs(save_model, exist_ok=True)

train_accs, train_losses = [], []
valid_accs, valid_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs+1):
    epoch_start_time = time.time()

    # Training
    train_acc, train_loss = train(lenet_model, optimizer, criterion, train_dataloader, device, epoch)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    # Evaluation
    eval_acc, eval_loss = evaluate(lenet_model, criterion, valid_dataloader)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    # Save best model
    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model.pt')

    # Print loss, acc end epoch
    print("-" *59)
    print(
        "| End of epoch {:3d} | Time {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3}"
        "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f}".format(
                                                                epoch,
                                                                time.time() - epoch_start_time,
                                                                train_acc,
                                                                train_loss,
                                                                eval_acc,
                                                                eval_loss
                                                                )
        )
    print("-" *59)

    # Load best model
    lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))
    lenet_model.eval()

| epoch   1 |    50/  211 batches | accuracy    0.715
| epoch   1 |   100/  211 batches | accuracy    0.890
| epoch   1 |   150/  211 batches | accuracy    0.920
| epoch   1 |   200/  211 batches | accuracy    0.939
-----------------------------------------------------------
| End of epoch   1 | Time 18.39s | Train Accuracy    0.460 | Train Loss    0.947| Valid Accuracy    0.188 | Valid Loss    0.948
-----------------------------------------------------------


<ipython-input-28-828fe5ae2987>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model.load_state_dict(torch.load(save_model + '/lenet_model.pt'))


| epoch   2 |    50/  211 batches | accuracy    0.952
| epoch   2 |   100/  211 batches | accuracy    0.960
| epoch   2 |   150/  211 batches | accuracy    0.965
| epoch   2 |   200/  211 batches | accuracy    0.967
-----------------------------------------------------------
| End of epoch   2 | Time 19.64s | Train Accuracy    0.127 | Train Loss    0.966| Valid Accuracy    0.117 | Valid Loss    0.968
-----------------------------------------------------------
| epoch   3 |    50/  211 batches | accuracy    0.970
| epoch   3 |   100/  211 batches | accuracy    0.975
| epoch   3 |   150/  211 batches | accuracy    0.974
| epoch   3 |   200/  211 batches | accuracy    0.976
-----------------------------------------------------------
| End of epoch   3 | Time 19.32s | Train Accuracy    0.085 | Train Loss    0.977| Valid Accuracy    0.091 | Valid Loss    0.973
-----------------------------------------------------------
| epoch   4 |    50/  211 batches | accuracy    0.980
| epoch   4 |   10

In [ ]:
# 5. Đánh giá mô hình trên tập test
test_data.transform = test_transforms

test_dataloader = data.DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=0)

test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
test_acc, test_loss

(0.05197122060926631, 0.9819)